In [1]:
#!pip install optuna

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os, optuna
import warnings
from PIL import Image
from tensorflow import keras

# from google.colab import drive
# drive.mount("/content/drive")

K = keras.backend
# Changing default dir
os.chdir("/Users/pedroteche/Documents/GitHub/maize-crop-diagnose/")
# optuna.logging.set_verbosity(optuna.logging.WARNING)

In [3]:
# !mkdir 'data'
# !cp -r 'drive/MyDrive/maize-crop-diagnose' 'data'

In [4]:
device_name = tf.test.gpu_device_name()
if device_name != "/device:GPU:0":
    raise SystemError("GPU device not found")
print("Found GPU at: {}".format(device_name))

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

Found GPU at: /device:GPU:0


In [5]:
class OneCycleScheduler(tf.keras.callbacks.Callback):
    def __init__(
        self,
        iterations,
        max_lr=1e-3,
        start_lr=None,
        start_mom=0.95,
        min_mom=0.85,
        last_iterations=None,
        last_lr=None,
    ):
        self.iterations = iterations
        self.max_lr = max_lr
        self.start_lr = start_lr or max_lr / 10
        self.start_mom = start_mom
        self.min_mom = min_mom
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_lr = last_lr or self.start_lr / 1000
        self.iteration = 0

    def _interpolate(self, iter1, iter2, lr1, lr2):
        return (lr2 - lr1) * (self.iteration - iter1) / (iter2 - iter1) + lr1

    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            lr = self._interpolate(0, self.half_iteration, self.start_lr, self.max_lr)
            mom = self._interpolate(
                0, self.half_iteration, self.start_mom, self.min_mom
            )
        elif self.iteration < 2 * self.half_iteration:
            lr = self._interpolate(
                self.half_iteration, 2 * self.half_iteration, self.max_lr, self.start_lr
            )
            mom = self._interpolate(
                self.half_iteration,
                2 * self.half_iteration,
                self.min_mom,
                self.start_mom,
            )
        else:
            lr = self._interpolate(
                2 * self.half_iteration, self.iterations, self.start_lr, self.last_lr
            )
            mom = self.start_mom
        self.iteration += 1
        K.set_value(self.model.optimizer.learning_rate, lr)
        K.set_value(self.model.optimizer.momentum, mom)


class OneCycleSchedulerNoMom(tf.keras.callbacks.Callback):
    def __init__(
        self,
        iterations,
        max_lr=1e-3,
        start_lr=None,
        last_iterations=None,
        last_lr=None,
    ):
        self.iterations = iterations
        self.max_lr = max_lr
        self.start_lr = start_lr or max_lr / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_lr = last_lr or self.start_lr / 1000
        self.iteration = 0

    def _interpolate(self, iter1, iter2, lr1, lr2):
        return (lr2 - lr1) * (self.iteration - iter1) / (iter2 - iter1) + lr1

    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            lr = self._interpolate(0, self.half_iteration, self.start_lr, self.max_lr)
        elif self.iteration < 2 * self.half_iteration:
            lr = self._interpolate(
                self.half_iteration, 2 * self.half_iteration, self.max_lr, self.start_lr
            )
        else:
            lr = self._interpolate(
                2 * self.half_iteration, self.iterations, self.start_lr, self.last_lr
            )
        self.iteration += 1
        K.set_value(self.model.optimizer.learning_rate, lr)


def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [6]:
IMG_HEIGHT = 64 * 2
IMG_WIDTH = 48 * 2
BATCH_SIZE = 32
EPOCH = 50
# DATA_DIR = "data/maize-crop-diagnose/data/train"
TRAIN_DATA_DIR = "/Volumes/DOCK-HD/Data/maize-crop-diagnose/train"
TEST_DATA_DIR = "/Volumes/DOCK-HD/Data/maize-crop-diagnose/test"

In [7]:

K.clear_session()
train_set, val_set = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DATA_DIR,
    validation_split=0.2,
    subset="both",
    seed=42,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
)
num_classes = len(train_set.class_names)
AUTOTUNE = tf.data.AUTOTUNE
print(train_set.cardinality().numpy())
train_set = train_set.prefetch(buffer_size=AUTOTUNE).cache()
val_set = val_set.prefetch(buffer_size=AUTOTUNE).cache()

Found 14749 files belonging to 3 classes.
Using 11800 files for training.
Using 2949 files for validation.
369


In [8]:
TRAIN_SIZE = 11800
TEST_SIZE = 2949

In [9]:
K.clear_session()
test_set= tf.keras.utils.image_dataset_from_directory(
    TEST_DATA_DIR,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
)
AUTOTUNE = tf.data.AUTOTUNE
test_set = test_set.prefetch(buffer_size=AUTOTUNE).cache()

Found 600 files belonging to 3 classes.


# Simple input/output example

In [10]:
input = keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
rescale = keras.layers.Rescaling(1.0 / 255)(input)
conv_in = keras.layers.Conv2D(
    filters=64, kernel_size=(7, 7), strides=(2, 2), activation="relu"
)(rescale)
pool_in = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv_in)

In [11]:
pool_out = keras.layers.GlobalAveragePooling2D()(pool_in)
dense_out = keras.layers.Dense(512, activation="relu")(pool_out)
output = keras.layers.Dense(num_classes, activation="softmax")(dense_out)

In [12]:
model = keras.Model(inputs=input, outputs=output)

In [13]:
model.compile(
    optimizer=tf.keras.optimizers.legacy.SGD(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

In [14]:
onecycle = OneCycleSchedulerNoMom(
    TRAIN_SIZE // BATCH_SIZE * EPOCH,
    max_lr=0.1,
    start_lr=0.01,
    last_lr=0.001,
)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

model.fit(
    train_set,
    validation_data=val_set,
    epochs=EPOCH,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping, onecycle],
)

369/369 [==============================] - 17s 44ms/step - loss: 1.0185 - accuracy: 0.4600 - val_loss: 0.9392 - val_accuracy: 0.5381
Epoch 2/50
369/369 [==============================] - 9s 23ms/step - loss: 0.8285 - accuracy: 0.6969 - val_loss: 0.6916 - val_accuracy: 0.7691
Epoch 3/50
369/369 [==============================] - 9s 23ms/step - loss: 0.6258 - accuracy: 0.7588 - val_loss: 0.6707 - val_accuracy: 0.6999
Epoch 4/50
369/369 [==============================] - 9s 23ms/step - loss: 0.5399 - accuracy: 0.7853 - val_loss: 0.6014 - val_accuracy: 0.7352
Epoch 5/50
369/369 [==============================] - 9s 25ms/step - loss: 0.4840 - accuracy: 0.8103 - val_loss: 0.5354 - val_accuracy: 0.7704
Epoch 6/50
369/369 [==============================] - 10s 28ms/step - loss: 0.4565 - accuracy: 0.8195 - val_loss: 0.5105 - val_accuracy: 0.7870
Epoch 7/50
369/369 [==============================] - 9s 24ms/step - loss: 0.4308 - accuracy: 0.8285 - val_loss: 0.4420 - val_accuracy: 0.8220
Epoch 8/

# One Residual Unit

In [15]:
K.clear_session()
input = keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
rescale = keras.layers.Rescaling(1.0 / 255)(input)
conv_in = keras.layers.Conv2D(
    filters=64, kernel_size=(7, 7), strides=(2, 2), activation="relu"
)(rescale)
pool_in = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding = "same")(conv_in)

In [16]:
conv_r1_1 = keras.layers.Conv2D(64, 3, 1, padding = "same")(pool_in)
bn_r1_1 = keras.layers.BatchNormalization()(conv_r1_1)
relu_r1_1 = keras.layers.ReLU()(bn_r1_1)
conv_r1_2 = keras.layers.Conv2D(64, 3, 1, padding = "same")(relu_r1_1)
bn_r1_2 = keras.layers.BatchNormalization()(conv_r1_2)
skip_r1 = keras.layers.Add()([bn_r1_2, pool_in])
relu_r1_2 = keras.layers.ReLU()(skip_r1)

In [17]:
pool_out = keras.layers.GlobalAveragePooling2D()(relu_r1_2)
dense_out = keras.layers.Dense(512, activation="relu")(pool_out)
output = keras.layers.Dense(num_classes, activation="softmax")(dense_out)

model = keras.Model(inputs=input, outputs=output)

In [18]:
onecycle = OneCycleSchedulerNoMom(
    TRAIN_SIZE // BATCH_SIZE * EPOCH,
    max_lr=0.1,
    start_lr=0.01,
    last_lr=0.001,
)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, start_from_epoch=20)

model.compile(
    optimizer=tf.keras.optimizers.legacy.SGD(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

model.fit(
    train_set,
    validation_data=val_set,
    epochs=EPOCH,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping, onecycle],
)

Epoch 1/50
369/369 [==============================] - 13s 35ms/step - loss: 0.4590 - accuracy: 0.8373 - val_loss: 0.5213 - val_accuracy: 0.8057
Epoch 2/50
369/369 [==============================] - 12s 33ms/step - loss: 0.3212 - accuracy: 0.8759 - val_loss: 0.3311 - val_accuracy: 0.8688
Epoch 3/50
369/369 [==============================] - 12s 33ms/step - loss: 0.2718 - accuracy: 0.8962 - val_loss: 0.5256 - val_accuracy: 0.8206
Epoch 4/50
369/369 [==============================] - 12s 33ms/step - loss: 0.2576 - accuracy: 0.9008 - val_loss: 0.2376 - val_accuracy: 0.9054
Epoch 5/50
369/369 [==============================] - 12s 33ms/step - loss: 0.2238 - accuracy: 0.9141 - val_loss: 0.3624 - val_accuracy: 0.8633
Epoch 6/50
369/369 [==============================] - 12s 33ms/step - loss: 0.1974 - accuracy: 0.9261 - val_loss: 0.4479 - val_accuracy: 0.8511
Epoch 7/50
369/369 [==============================] - 12s 33ms/step - loss: 0.1790 - accuracy: 0.9344 - val_loss: 0.3635 - val_accuracy:

# 2 Residual Units

In [25]:
K.clear_session()
# Input layers
input = keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
rescale = keras.layers.Rescaling(1.0 / 255)(input)
conv_in = keras.layers.Conv2D(
    filters=64, kernel_size=(7, 7), strides=(2, 2), activation="relu"
)(rescale)
pool_in = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding = "same")(conv_in)
# R1
conv_r1_1 = keras.layers.Conv2D(64, 3, 1, padding = "same")(pool_in)
bn_r1_1 = keras.layers.BatchNormalization()(conv_r1_1)
relu_r1_1 = keras.layers.ReLU()(bn_r1_1)
conv_r1_2 = keras.layers.Conv2D(64, 3, 1, padding = "same")(relu_r1_1)
bn_r1_2 = keras.layers.BatchNormalization()(conv_r1_2)
skip_r1 = keras.layers.Add()([bn_r1_2, pool_in])
relu_r1_2 = keras.layers.ReLU()(skip_r1)
# R2
conv_r2_1 = keras.layers.Conv2D(64, 3, 1, padding = "same")(relu_r1_2)
bn_r2_1 = keras.layers.BatchNormalization()(conv_r2_1)
relu_r2_1 = keras.layers.ReLU()(bn_r2_1)
conv_r2_2 = keras.layers.Conv2D(64, 3, 1, padding = "same")(relu_r2_1)
bn_r2_2 = keras.layers.BatchNormalization()(conv_r2_2)
skip_r2 = keras.layers.Add()([bn_r2_2, conv_r2_1])
relu_r2_2 = keras.layers.ReLU()(skip_r2)
# Output layers
pool_out = keras.layers.GlobalAveragePooling2D()(relu_r2_2)
dense_out = keras.layers.Dense(512, activation="relu")(pool_out)
output = keras.layers.Dense(num_classes, activation="softmax")(dense_out)
# Model
model = keras.Model(inputs=input, outputs=output)

In [26]:
onecycle = OneCycleSchedulerNoMom(
    TRAIN_SIZE // BATCH_SIZE * EPOCH,
    max_lr=0.1,
    start_lr=0.01,
    last_lr=0.001,
)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, start_from_epoch=20)

model.compile(
    optimizer=tf.keras.optimizers.legacy.SGD(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

model.fit(
    train_set,
    validation_data=val_set,
    epochs=EPOCH,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping, onecycle],
)

Epoch 1/50
369/369 [==============================] - 22s 57ms/step - loss: 0.4148 - accuracy: 0.8431 - val_loss: 0.5897 - val_accuracy: 0.7596
Epoch 2/50
369/369 [==============================] - 16s 43ms/step - loss: 0.3193 - accuracy: 0.8777 - val_loss: 0.3175 - val_accuracy: 0.8772
Epoch 3/50
369/369 [==============================] - 16s 43ms/step - loss: 0.2698 - accuracy: 0.8962 - val_loss: 0.4354 - val_accuracy: 0.8532
Epoch 4/50
369/369 [==============================] - 16s 43ms/step - loss: 0.2257 - accuracy: 0.9154 - val_loss: 0.2962 - val_accuracy: 0.8922
Epoch 5/50
369/369 [==============================] - 16s 43ms/step - loss: 0.1965 - accuracy: 0.9277 - val_loss: 0.2653 - val_accuracy: 0.9047
Epoch 6/50
369/369 [==============================] - 16s 44ms/step - loss: 0.1754 - accuracy: 0.9368 - val_loss: 0.1877 - val_accuracy: 0.9376
Epoch 7/50
369/369 [==============================] - 16s 43ms/step - loss: 0.1583 - accuracy: 0.9438 - val_loss: 0.1370 - val_accuracy:

# 3 Residual Units

In [27]:
K.clear_session()
# Input layers
input = keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
rescale = keras.layers.Rescaling(1.0 / 255)(input)
conv_in = keras.layers.Conv2D(
    filters=64, kernel_size=(7, 7), strides=(2, 2), activation="relu"
)(rescale)
pool_in = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding = "same")(conv_in)
# R1
conv_r1_1 = keras.layers.Conv2D(64, 3, 1, padding = "same")(pool_in)
bn_r1_1 = keras.layers.BatchNormalization()(conv_r1_1)
relu_r1_1 = keras.layers.ReLU()(bn_r1_1)
conv_r1_2 = keras.layers.Conv2D(64, 3, 1, padding = "same")(relu_r1_1)
bn_r1_2 = keras.layers.BatchNormalization()(conv_r1_2)
skip_r1 = keras.layers.Add()([bn_r1_2, pool_in])
relu_r1_2 = keras.layers.ReLU()(skip_r1)
# R2
conv_r2_1 = keras.layers.Conv2D(64, 3, 1, padding = "same")(relu_r1_2)
bn_r2_1 = keras.layers.BatchNormalization()(conv_r2_1)
relu_r2_1 = keras.layers.ReLU()(bn_r2_1)
conv_r2_2 = keras.layers.Conv2D(64, 3, 1, padding = "same")(relu_r2_1)
bn_r2_2 = keras.layers.BatchNormalization()(conv_r2_2)
skip_r2 = keras.layers.Add()([bn_r2_2, relu_r1_2])
relu_r2_2 = keras.layers.ReLU()(skip_r2)
# R3
conv_r3_skip = keras.layers.Conv2D(128, 1, 2, padding = "same")(relu_r2_2)
conv_r3_1 = keras.layers.Conv2D(128, 3, 2, padding = "same")(relu_r2_2)
bn_r3_1 = keras.layers.BatchNormalization()(conv_r3_1)
relu_r3_1 = keras.layers.ReLU()(bn_r3_1)
conv_r3_2 = keras.layers.Conv2D(128, 3, 1, padding = "same")(relu_r3_1)
bn_r3_2 = keras.layers.BatchNormalization()(conv_r3_2)
skip_r3 = keras.layers.Add()([bn_r3_2, conv_r3_skip])
relu_r3_2 = keras.layers.ReLU()(skip_r2)
# Output layers
pool_out = keras.layers.GlobalAveragePooling2D()(relu_r2_2)
dense_out = keras.layers.Dense(512, activation="relu")(pool_out)
output = keras.layers.Dense(num_classes, activation="softmax")(dense_out)
# Model
model = keras.Model(inputs=input, outputs=output)

In [28]:
onecycle = OneCycleSchedulerNoMom(
    TRAIN_SIZE // BATCH_SIZE * EPOCH,
    max_lr=0.1,
    start_lr=0.01,
    last_lr=0.001,
)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, start_from_epoch=20)

model.compile(
    optimizer=tf.keras.optimizers.legacy.SGD(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

model.fit(
    train_set,
    validation_data=val_set,
    epochs=EPOCH,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping, onecycle],
)

Epoch 1/50
369/369 [==============================] - 23s 61ms/step - loss: 0.4100 - accuracy: 0.8442 - val_loss: 0.4650 - val_accuracy: 0.8345
Epoch 2/50
369/369 [==============================] - 16s 43ms/step - loss: 0.2800 - accuracy: 0.8940 - val_loss: 0.3555 - val_accuracy: 0.8674
Epoch 3/50
369/369 [==============================] - 16s 43ms/step - loss: 0.2289 - accuracy: 0.9140 - val_loss: 0.8192 - val_accuracy: 0.6924
Epoch 4/50
369/369 [==============================] - 16s 43ms/step - loss: 0.2370 - accuracy: 0.9117 - val_loss: 0.8163 - val_accuracy: 0.7362
Epoch 5/50
369/369 [==============================] - 16s 43ms/step - loss: 0.2049 - accuracy: 0.9240 - val_loss: 0.6976 - val_accuracy: 0.8047
Epoch 6/50
369/369 [==============================] - 16s 43ms/step - loss: 0.1698 - accuracy: 0.9381 - val_loss: 0.8491 - val_accuracy: 0.7586
Epoch 7/50
369/369 [==============================] - 16s 43ms/step - loss: 0.1535 - accuracy: 0.9445 - val_loss: 1.2112 - val_accuracy:

# Optimization

In [ ]:
def objective(trial):
    K.clear_session()
    train_set, val_set = tf.keras.utils.image_dataset_from_directory(
        DATA_DIR,
        validation_split=0.2,
        subset="both",
        seed=42,
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
    )
    num_classes = len(train_set.class_names)
    AUTOTUNE = tf.data.AUTOTUNE
    train_set = train_set.prefetch(buffer_size=AUTOTUNE).cache()
    val_set = val_set.prefetch(buffer_size=AUTOTUNE).cache()

    # Network architecture
    ## Input
    ### Hyperparameters for input
    input_kernel_size = trial.suggest_categorical("input_kernel_size", [3, 5, 7])
    ### Architecture
    input_layer = keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    conv_layer_1 = keras.layers.Conv2D(
        filters=32, kernel_size=input_kernel_size, activation="relu"
    )(input_layer)
    pool_layer_1 = keras.layers.MaxPooling2D()(conv_layer_1)
    ## Residual Layers
    ### Hyperparameters for Residual
    residual_kernel_size = trial.suggest_categorical("residual_kernel_size", [3, 5, 7])
    ### Architecture
    #### R1
    r1_conv_layer1 = keras.layers.Conv2D(
        filters=32,
        kernel_size=residual_kernel_size,
        activation="relu",
        padding="same",
    )(pool_layer_1)
    r1_batch_norm = keras.layers.BatchNormalization()(r1_conv_layer1)
    r1_conv_layer2 = keras.layers.Conv2D(
        filters=32, kernel_size=residual_kernel_size, padding="same"
    )(r1_batch_norm)
    r1_batch_norm_2 = keras.layers.BatchNormalization()(r1_conv_layer2)
    r1_out = keras.layers.Add()([pool_layer_1, r1_batch_norm_2])
    r1_relu = keras.layers.ReLU()(r1_out)
    #### R2
    r2_conv_layer1 = keras.layers.Conv2D(
        filters=32,
        kernel_size=residual_kernel_size,
        activation="relu",
        padding="same",
    )(r1_relu)
    r2_batch_norm = keras.layers.BatchNormalization()(r2_conv_layer1)
    r2_conv_layer2 = keras.layers.Conv2D(
        filters=32, kernel_size=residual_kernel_size, padding="same"
    )(r2_batch_norm)
    r2_batch_norm_2 = keras.layers.BatchNormalization()(r2_conv_layer2)
    r2_out = keras.layers.Add()([r1_relu, r2_batch_norm_2])
    r2_relu = keras.layers.ReLU()(r2_out)
    #### R3
    r3_conv_layer1 = keras.layers.Conv2D(
        filters=64,
        strides=2,
        kernel_size=residual_kernel_size,
        activation="relu",
        padding="same",
    )(r2_relu)
    r3_batch_norm = keras.layers.BatchNormalization()(r3_conv_layer1)
    r3_conv_layer2 = keras.layers.Conv2D(
        filters=64, kernel_size=residual_kernel_size, padding="same"
    )(r3_batch_norm)
    r3_batch_norm_2 = keras.layers.BatchNormalization()(r3_conv_layer2)
    r3_conv_layer_inter = keras.layers.Conv2D(
        filters=64,
        strides=2,
        kernel_size=1,
        padding="same",
    )(r2_relu)
    r3_out = keras.layers.Add()([r3_conv_layer_inter, r3_batch_norm_2])
    r3_relu = keras.layers.ReLU()(r3_out)

    ## Output
    ### Hyperparameters for Output
    num_output = trial.suggest_int("num_input_filter", 16, 128)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)
    ### Architecture
    pool = keras.layers.MaxPooling2D()(r3_relu)
    flatten = keras.layers.Flatten()(pool)
    dense = keras.layers.Dense(num_output, activation="relu")(flatten)
    dropout = keras.layers.Dropout(dropout_rate)(dense)
    output = keras.layers.Dense(num_classes, activation="softmax")(dropout)
    # Model definition
    model = keras.Model(inputs=input_layer, outputs=output)

    # Hyperparameters for OneCycleScheduler
    max_lr = trial.suggest_float("max_lr", 0.005, 0.5)
    start_lr = trial.suggest_float("start_lr", max_lr * 0.01, max_lr * 0.8)
    last_lr = trial.suggest_float("last_lr", start_lr, max_lr)
    # Model definition
    onecycle = OneCycleSchedulerNoMom(
        TRAIN_SIZE // BATCH_SIZE * EPOCH,
        max_lr=max_lr,
        start_lr=start_lr,
        last_lr=last_lr,
    )
    early_stopping = tf.keras.callbacks.EarlyStopping(
        patience=5, restore_best_weights=True
    )
    model.compile(
        optimizer=tf.keras.optimizers.SGD(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=["accuracy"],
    )
    # Fitting model
    history = model.fit(
        train_set,
        validation_data=val_set,
        epochs=EPOCH,
        batch_size=BATCH_SIZE,
        callbacks=[early_stopping, onecycle],
    )
    # Evaluating and returning F1 score
    loss, acc = model.evaluate(val_set)
    return acc


study = optuna.create_study(
    storage="sqlite:///drive/MyDrive/maize-crop-diagnose/db_maize_models.sqlite3",
    study_name="resnet_onecycle_sgd",
    direction="maximize",
    load_if_exists=True,
)

study.optimize(objective, n_trials=100)
print(f"Best value: {study.best_value} (params: {study.best_params})")